# Random Variable Neighborhood Search (RVNS) for Hub Location

This notebook aims to implement and analyze the Random Variable Neighborhood Search (RVNS) algorithm for solving the Uncapacitated Single Allocation p-Hub Median Problem (USApHMP). In the USApHMP, we are given a network with nodes and edges, where each node represents a potential location for a hub. The task is to select p nodes to place the hubs in such a way that the total transportation cost is minimized.

The RVNS algorithm is a metaheuristic approach that explores different "neighborhoods" (i.e., variations) of the current solution, aiming to find improvements. This process is repeated for a specified number of iterations, or until a satisfactory solution is found. The RVNS has two main versions: the RVNS with the first improvement strategy and the RVNS with the best improvement strategy.

The RVNS with the first improvement strategy generates a neighboring solution and accepts it if it improves upon the current solution. The process is then repeated from this new solution. If the new solution is not an improvement, a new neighboring solution is generated, and the process continues.

The RVNS with the best improvement strategy, on the other hand, generates all possible neighboring solutions and selects the best among them. If this new solution is better than the current one, it replaces the current solution, and the process repeats.


In [ ]:
# load the data
%run data_preparation.ipynb

In [ ]:
# import libraries
import time
import csv
import matplotlib.pyplot as plt
import numpy as np

# load the functions
from hub_operations import neighbour, neighbour2, neighbour3, neighbour4, initial_solution, total_cost

In [ ]:
# Store the matrices in a dictionary
matrices = {
    'CAB_10_fm': CAB_10_fm,
    'CAB_10_cm': CAB_10_cm,
    'CAB_25_fm': CAB_25_fm,
    'CAB_25_cm': CAB_25_cm,
    'TR_55_fm': TR_55_fm,
    'TR_55_cm': TR_55_cm,
    'TR_81_fm': TR_81_fm,
    'TR_81_cm': TR_81_cm,
    'RGP_100_fm': RGP_100_fm,
    'RGP_100_cm': RGP_100_cm
}

rvns_results_file = 'RVNS Results.csv'

with open(rvns_results_file, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['RVNS Algorithm', 'fm_name', 'cm_name', 'number_of_hubs', 'best_cost', 'current_solution', 'best_costs', 'best_solution', 'total_time'])


In [ ]:
# function for RVNS with best improvement
def RVNS_best_improvement(fm_name, cm_name, number_of_hubs, max_iter):
    start_time = time.perf_counter()
    # generate initial solution 
    current_solution = initial_solution(matrices[fm_name], matrices[cm_name], number_of_hubs)
    best_cost = [total_cost(matrices[fm_name], matrices[cm_name], current_solution)]
    best_solution = current_solution.copy()
    
    neighbourhood_structures = [neighbour, neighbour2, neighbour3, neighbour4]
    neighbour_cost = [0, 0, 0, 0]
    neighbour_solution = [0, 0, 0, 0]
    for _ in range(max_iter):
        # generate a neighbour solution from current solution using neighbourhood_structures[k]
        for k, neighbour_func in enumerate(neighbourhood_structures):
            neighbour_solution[k] = neighbour_func(current_solution)
            neighbour_cost[k] = total_cost(matrices[fm_name], matrices[cm_name], neighbour_solution[k])
            
        # find the best neighbour solution
        best_neighbour_cost = min(neighbour_cost)

        if best_neighbour_cost < best_cost[-1]:
            current_solution = neighbour_solution[neighbour_cost.index(best_neighbour_cost)]
            best_solution = current_solution.copy()
            best_cost.append(best_neighbour_cost)
        else:
            best_cost.append(best_cost[-1])

    total_time = time.perf_counter() - start_time

    with open(rvns_results_file, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(['RVNS_best_improvement', fm_name, cm_name, number_of_hubs, best_cost[-1], current_solution, best_cost, best_solution, total_time])

    return best_cost, best_solution, total_time

# function for RVNS with first improvement
def RVNS_first_improvement(fm_name, cm_name, number_of_hubs, max_iter):
    start_time = time.perf_counter()
    # generate initial solution 
    current_solution = initial_solution(matrices[fm_name], matrices[cm_name], number_of_hubs)
    best_cost = [total_cost(matrices[fm_name], matrices[cm_name], current_solution)]
    
    neighbourhood_structures = [neighbour, neighbour2, neighbour3, neighbour4]
    t = 0
    while t < max_iter:
        k = 1
        # generate a neighbour solution from current solution using neighbourhood_structures[k]
        while k <= len(neighbourhood_structures):
            neighbour_solution = neighbourhood_structures[k-1](current_solution)
            neighbour_cost = total_cost(matrices[fm_name], matrices[cm_name], neighbour_solution)
            
            if neighbour_cost < best_cost[-1]:
                current_solution = neighbour_solution
                best_cost.append(neighbour_cost)
                k = 1
            else:
                best_cost.append(best_cost[-1])
                k += 1
            t += 1

    total_time = time.perf_counter() - start_time

    with open(rvns_results_file, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(['RVNS_first_improvement', fm_name, cm_name, number_of_hubs, best_cost[-1], current_solution, best_cost, total_time])

    return best_cost, current_solution, total_time


In [ ]:
# final run for RVNS
fm_names = ['CAB_10_fm', 'CAB_25_fm', 'TR_55_fm', 'TR_81_fm', 'RGP_100_fm']
number_of_hubs_dict = {'CAB_10_fm': [3], 'CAB_25_fm': [3,5], 'TR_55_fm': [3,5], 'TR_81_fm': [5,7], 'RGP_100_fm': [7,10]}
max_ts_dict = {'CAB_10_fm': [500], 'CAB_25_fm': [2000], 'TR_55_fm': [2500], 'TR_81_fm': [5000], 'RGP_100_fm': [6000]}

# run 10 times
for i in range(10):
    print(f'Run: {i}')
    for fm_name in fm_names:
        cm_name = fm_name.replace('fm', 'cm')
        for number_of_hubs in number_of_hubs_dict[fm_name]:
            for max_t in max_ts_dict[fm_name]:
                print(f'fm_name: {fm_name}, cm_name: {cm_name}, number_of_hubs: {number_of_hubs}, max_t: {max_t}')
                # run rvns
                best_cost, best_solution, total_time = RVNS_best_improvement(fm_name, cm_name, number_of_hubs, max_t)
                print(f'best_cost: {best_cost[-1]}')
                print(f'Timer ended. Total time: {total_time}')
                # plot the best cost
                plt.plot(best_cost)
                plt.show()
                print('Run completed.')
                print('--------------------------------------------------------------------------------------------------------------------')
